# Bias Uncovering Test Case (BTC) for Gender Bias

In [1]:
import os
import pandas as pd
import numpy as np
import math
import time
import pickle

from metric import evaluate_btc, evaluate_fairness_violation


## Measuring the Performance of the Fine-tuned SA models on the Test set

In [2]:
def load_pickle(fpath):
    with open(fpath, 'rb') as f:
        pred = pickle.load(f)
    return pred

def accuracy(label, prediction):
    return round(100 * sum(label == prediction) / len(label), 2)

def calculate_test_accuracy(task, model) :
    label_path = f"../../asset/{task}/test.csv"
    pred_path = f"../../asset/{task}/predictions/{model}.pkl"


    test_df = pd.read_csv(label_path, header=None, sep="\t")

    test_labels = test_df[0].values
    predicitons = load_pickle(pred_path)

    return accuracy(test_labels, predicitons)


task = "imdb"
model = "bert-base-uncased"


calculate_test_accuracy(task, model)


92.57

## Evaluating the Performance of fine-tuned models

In [3]:
models = ["bert-base-uncased", "bert-base-cased", "roberta-base", "xlnet-base-cased",
          "albert-base-v2", "microsoft/mpnet-base", "microsoft/deberta-base",
          "facebook/muppet-roberta-base", "google/electra-base-generator"]

task = "imdb"  # dataset used for fine-tuning
mutant = "imdb"  # dataset used for generating mutants

df = pd.DataFrame(columns=["model", "accuracy"])

for model in models:
    test_accuracy = calculate_test_accuracy(task, model)
    df = df.append(
        {
            "model": model,
            "accuracy": test_accuracy
        },
        ignore_index=True)

df

,model,accuracy
0,bert-base-uncased,92.57
1,bert-base-cased,89.12
2,roberta-base,92.84
3,xlnet-base-cased,93.88
4,albert-base-v2,89.45
5,microsoft/mpnet-base,93.62
6,microsoft/deberta-base,93.42
7,facebook/muppet-roberta-base,95.29
8,google/electra-base-generator,91.20


In [4]:
mutation_tool = "biasfinder"
bias_type = "gender"
task = "imdb"  # dataset used for fine-tuning
model = "bert-base-uncased"
mutant = "imdb"  # dataset used for generating mutants

def load_original_prediction(mutation_tool, bias_type, mutant):
    base_dir = f"../../data/{mutation_tool}/{bias_type}/{mutant}/"
    ori_df = pd.read_csv(base_dir + "original.csv", header=None, sep="\t",
                         names=["label", "original"])
    original_prediction_fpath = os.path.join(
        base_dir, f"original-predictions/{model}.pkl")
    ori_df["prediction"] = load_pickle(original_prediction_fpath)
    return ori_df


ori_df = load_original_prediction(mutation_tool, bias_type, mutant)
ori_df


,label,original,prediction
0,0,"I hated the book. A guy meets a smart dog, get...",0
1,1,The Great Dictator is a beyondexcellent film. ...,1
2,1,In the groovy mid 70's a scruffy bunch of bras...,1
3,0,"If you are like me and observed the original ""...",0
4,1,The debut that plucked from obscurity one of t...,1
...,...,...,...
3010,1,"Well, the movie did turn out a lot better than...",1
3011,0,... Said the continuity announcer as TACTICAL ...,0
3012,0,"In this film, there is a loose plot of a man (...",0
3013,0,The weakest of the ' old ' crew Star Trek film...,0


## Measuring the BTC that can be found by BiasFinder

In [5]:
def load_mutant_and_prediction(mutation_tool, bias_type, mutant):
    base_dir = f"../../data/{mutation_tool}/{bias_type}/{mutant}/"
    if mutation_tool == "biasfinder" :
        headers = ["label", "mutant", "template", "original", "gender"]
    elif mutation_tool == "eec":
        headers = ["label", "mutant", "template", "original", "person", "gender", "emotion"]
    elif mutation_tool == "mtnlp":
        headers = ["label", "mutant", "original", "mutation_type"]
    else :
        raise ValueError("Unknown mutation tool")

    df = pd.read_csv(base_dir + "test.csv", header=None, sep="\t",names=headers)

    if mutation_tool == "biasfinder" or mutation_tool == "eec" :
        df["template"] = df["template"].astype("category")
        df["template_id"] = df["template"].cat.codes
    
    mutant_prediction_fpath = os.path.join(base_dir, f"mutant-predictions/{model}.pkl")
    
    df["prediction"] = load_pickle(mutant_prediction_fpath)

    ori_df = load_original_prediction(mutation_tool, bias_type, mutant)
    ori2prediction = {}
    for index, row in ori_df.iterrows():
        prediction = row["prediction"]
        text = row["original"]
        ori2prediction[text] = prediction
    
    df["original_prediction"] = df["original"].apply(lambda text: ori2prediction[text])

    
    return df


df = load_mutant_and_prediction(mutation_tool, bias_type, mutant)

df.head()


,label,mutant,template,original,gender,template_id,prediction,original_prediction
0,0,"I hated the book. A guy meets a smart dog, get...","I hated the book. A guy meets a smart dog, get...","I hated the book. A guy meets a smart dog, get...",male,929,0,0
1,0,"I hated the book. A guy meets a smart dog, get...","I hated the book. A guy meets a smart dog, get...","I hated the book. A guy meets a smart dog, get...",female,929,0,0
2,1,The Great Dictator is a beyondexcellent film. ...,The Great Dictator is a beyondexcellent film. ...,The Great Dictator is a beyondexcellent film. ...,male,1980,1,1
3,1,The Great Dictator is a beyondexcellent film. ...,The Great Dictator is a beyondexcellent film. ...,The Great Dictator is a beyondexcellent film. ...,male,1980,1,1
4,1,The Great Dictator is a beyondexcellent film. ...,The Great Dictator is a beyondexcellent film. ...,The Great Dictator is a beyondexcellent film. ...,male,1980,1,1


In [6]:
print("Accuracy on mutants: {:.2f}%".format(accuracy(df["label"],df["prediction"])))


Accuracy on mutants: 93.29%


In [7]:
def print_evaluation(evaluation):
    print("# Mutants \t:", evaluation["mutant"])
    print("# Templates \t:", evaluation["template"])
    print("# BTCs \t\t:", evaluation["btc"])

evaluation = evaluate_btc(df["label"], df["prediction"], df["mutant"], df["template"], "gender", df["gender"])
print_evaluation(evaluation)

# Mutants 	: 153866
# Templates 	: 3015
# BTCs 		: 5543


In [8]:
# models = ["bert-base-uncased", "bert-base-cased", "roberta-base", "albert-base-v2", "microsoft/mpnet-base", "microsoft/deberta-base", "facebook/muppet-roberta-base", "google/electra-base-generator"]

# fine-tune
# 

# test accuracy
# 

# predict mutants
# "roberta-base", "albert-base-v2",


models = ["bert-base-uncased"]

# models = ["bert-base-uncased", "bert-base-cased", "roberta-base", "xlnet-base-cased",
#           "albert-base-v2", "microsoft/mpnet-base", "microsoft/deberta-base",
#           "facebook/muppet-roberta-base", "google/electra-base-generator"]

mutation_tool = "biasfinder"
bias_type = "gender"
task = "imdb"  # dataset used for fine-tuning
mutant = "imdb"  # dataset used for generating mutants

d = pd.DataFrame(columns=["tool", "model", "accuracy",
                          "template", "mutant", "btc"])

for model in models :
    df = load_mutant_and_prediction(mutation_tool, bias_type, mutant)
    evaluation = evaluate_btc(df["label"], df["prediction"],
                            df["mutant"], df["template"], "gender", df["gender"])
    
    d = d.append(
            {
                "model": model, 
                "tool": mutation_tool, 
                "accuracy": calculate_test_accuracy(task, model),
                "template": evaluation['template'], 
                "mutant": evaluation['mutant'],
                "btc" : evaluation['btc']
            }, 
            ignore_index=True)


In [9]:
d

,tool,model,accuracy,template,mutant,btc
0,biasfinder,bert-base-uncased,92.57,3015,153866,5543


## Fairness Violation

In [10]:

# models = ["bert-base-uncased", "bert-base-cased", "roberta-base", "albert-base-v2", "microsoft/mpnet-base", "microsoft/deberta-base", "facebook/muppet-roberta-base", "google/electra-base-generator"]

# fine-tune
#

# test accuracy
#

# predict mutants
# "roberta-base", "albert-base-v2",


models = ["bert-base-uncased"]

# models = ["bert-base-uncased", "bert-base-cased", "roberta-base", "xlnet-base-cased",
#           "albert-base-v2", "microsoft/mpnet-base", "microsoft/deberta-base",
#           "facebook/muppet-roberta-base", "google/electra-base-generator"]

mutation_tool = "biasfinder"
bias_type = "gender"
task = "imdb"  # dataset used for fine-tuning
mutant = "imdb"  # dataset used for generating mutants

d = pd.DataFrame(columns=["tool", "model", "accuracy", "mutant", "fairness_violation"])

for model in models:
    df = load_mutant_and_prediction(mutation_tool, bias_type, mutant)
    fairness_violation = evaluate_fairness_violation(
        df["prediction"], df["original_prediction"])

    d = d.append(
        {
            "model": model,
            "tool": mutation_tool,
            "accuracy": calculate_test_accuracy(task, model),
            "mutant": len(df),
            "fairness_violation": fairness_violation
        },
        ignore_index=True)

mutation_tool = "mtnlp"
bias_type = ""
for model in models:
    df = load_mutant_and_prediction(mutation_tool, bias_type, mutant)
    fairness_violation = evaluate_fairness_violation(
        df["prediction"], df["original_prediction"])

    d = d.append(
        {
            "model": model,
            "tool": mutation_tool,
            "accuracy": calculate_test_accuracy(task, model),
            "mutant": evaluation['mutant'],
            "fairness_violation": fairness_violation
        },
        ignore_index=True)


In [11]:
d

,tool,model,accuracy,template,mutant,fairness_violation
0,biasfinder,bert-base-uncased,92.57,3015,153866,677
1,mtnlp,bert-base-uncased,92.57,3015,153866,1816
